In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("data/TAQ_30Min_AAPL_2023_normalized.csv")
df.index = df.datetime
df["Log_Turnover"] = np.log(df["Normalized_TURNOVER"])
T = 13
df.info()

In [ ]:
class Params:
    def __init__(self, pi, Sigma, a_eta, a_mu, sigma_eta_sq, sigma_mu_sq, r, phi, T=1):
        # pi and Sigma go into $x_t ~ \mathcal{N}(\pi_t, \Sigma_t)$
        self.pi = pi
        self.Sigma = Sigma
        # a_eta and a_mu define the state transition matrix A = [a_eta 0; 0 a_mu]
        self.a_eta = a_eta
        self.a_mu = a_mu
        # sigma_eta and sigma_mu define the covariance matrix Q = [sigma_eta^2 0; 0 sigma_mu^2]
        # for the Gaussian noise in the state transition w_t ~ \mathcal{N}(0, Q_t)
        self.sigma_eta_sq = sigma_eta_sq
        self.sigma_mu_sq = sigma_mu_sq
        # r goes into v_t ~ \mathcal{N}(0,r) where v_t is the noise in observation t
        self.r = r
        # phi is the seasonality parameter.
        # It's a vector in $\mathbb{R}^T$ where T is the number of intraday observations in a day
        self.phi = phi
        self.T = T
    
    def A(self, tau):
        a1 = 1.0
        a2 = 1.0
        T = self.T
        if tau % T == 0: # tau = kT for some integer K, T is the # of observations in a day
            a1 = self.a_eta
            a2 = self.a_mu
        return np.vstack([np.hstack([np.eye(T)*a1, np.zeros((T,T))]),
                         np.hstack([np.zeros((T,T)), np.eye(T)*a2])])
    
    def Q(self, tau):
        a1 = 0.0
        a2 = 0.0
        T = self.T
        if tau % T == 0: # tau = kT for some integer K, T is the # of observations in a day
            a1 = self.sigma_eta_sq
            a2 = self.sigma_mu_sq
        return np.vstack([np.hstack([np.eye(T)*a1, np.zeros((T,T))]),
                          np.hstack([np.zeros((T,T)), np.eye(T)*a2])])

In [ ]:
C = np.hstack([np.eye(T), np.eye(T)])
C.shape

In [ ]:
# test
theta = Params(np.zeros(2), np.identity(2)*0.5, 1.0, 1.0, 0.0025, 0.0025, 0.0005, np.array([0.6, 0.25, 0.0, -0.15, -0.3, -0.45, -0.5, -0.6, -0.5, -0.25, -0.3, -0.1, 0.4]), T=13)

## Expectation maximization
In this step we want to predict $x_\tau = [\eta_\tau\ \mu_\tau]^\top \in \mathbb{R}^2$ which is the hidden state vector. The variables $\eta_\tau$ and $\mu_\tau$ are the daily average and intraday dynamic part of the log volume.

In [ ]:
def kalman_filtering(tau, x_hat_tau, y_tau_plus, Sigma_tau_tau, params):
    A = params.A(tau)
    x_hat_tau_plus = A @ x_hat_tau # predict mean
    Sigma_tau_plus = A @ Sigma_tau_tau @ A.T + params.Q(tau) # predict covariance
    
    # compute Kalman gain
    K_tau_plus = Sigma_tau_plus @ C.T @ np.linalg.inv(C @ Sigma_tau_plus @ C.T + params.r)
    
    # correct conditional mean
    x_hat_next = x_hat_tau_plus + K_tau_plus @ (y_tau_plus - params.phi - C@x_hat_tau_plus)
    Sigma_next = Sigma_tau_plus - K_tau_plus @ C @ Sigma_tau_plus
    #print("x_hat_next", x_hat_next.shape, "Sigma_next", Sigma_next.shape)
    return x_hat_next, Sigma_next

In [ ]:
# set up and run a dimensional test
y_1 = df.head(T)["Log_Turnover"]
x_1 = np.reshape(np.array([y_1/2, y_1/2]), 2*T)
Sigma_1 = np.eye(2*T)
x_plus, Sigma_plus = kalman_filtering(1, x_1, y_1, Sigma_1, theta)
print("Shape should be {}: x.shape = {}".format(2*T, x_plus.shape))
print("Shape should be {} x {}: Sigma.shape = {}".format(2*T, 2*T, Sigma_plus.shape))

In [ ]:
def kalman_smoothing(x_t, ys, Sigma_t, params):
    # this uses the outputs from the filtering algorithm
    # NOTE THAT x_t is a shorthand in the next few lines for x_{t|t} and Sigma_t := Sigma_{t|t}
    N = ys.shape[0]
    x_ts = []
    Sigma_ts = []
    
    # this is an unsightly way to code it but I think it makes more sense
    for t in range(0, N):
        x_t, Sigma_t = kalman_filtering(t, x_t, ys[t], Sigma_t, params)
        x_ts.append(x_t)
        Sigma_ts.append(Sigma_t)
        
    x_N, Sigma_N = x_ts[-1], Sigma_ts[-1]
    # Now we have x_{N|N}, Sigma_{N|N}
    
    x_tau_n = x_N # this is the initialization of x_{t+1|N} and Sigma_{t+1|N}
    Sigma_tau_n = Sigma_N
    Lt = np.zeros_like(Sigma_t)
    
    for t in range(N-1, 0, -1):
        A = params.A(t)
        # in here, Sigma_ts[t-1] is Sigma_{t|t} because of 0-indexing
        Sigma_tau_plus = A @ Sigma_ts[t-1] @ A.T + params.Q(t)
        x_hat_tau_plus = A @ x_ts[t-1]
        
        Lt = Sigma_ts[t-1] @ A.T @ np.linalg.inv(Sigma_tau_plus)
        x_tau_n = x_ts[t-1] + Lt @ (x_tau_n - x_hat_tau_plus)
        Sigma_tau_n = Sigma_ts[t-1] + Lt @ (Sigma_tau_n - Sigma_tau_plus) @ Lt.T
    return x_tau_n, Sigma_tau_n, Lt, x_ts, Sigma_ts # this is x_{t|N} and Sigma_{t|N}

In [ ]:
ys = df["Log_Turnover"].to_numpy()
#ys = np.reshape(ys, (int(ys.shape[0]/T), T)) # reshape ys to (N_days, T)

N_train = T*10

x_tau_n, Sigma_tau_n, _, _, _ = kalman_smoothing(x_1, ys[0:N_train], Sigma_1, theta)
# dimensional check again
print(x_tau_n.shape)
print(Sigma_tau_n.shape)
plt.imshow(Sigma_tau_n)

In [ ]:
def compare(params1, params2):
    err = np.mean(np.abs(params1.pi - params2.pi)) + np.mean(np.abs(params1.Sigma - params2.Sigma)) + \
          np.abs(params1.a_eta - params2.a_eta) + np.abs(params1.a_mu - params2.a_mu) + \
          np.abs(params1.sigma_eta_sq - params2.sigma_eta_sq) + \
          np.abs(params1.sigma_mu_sq - params2.sigma_mu_sq) + \
          np.abs(params1.r - params2.r) + np.mean(np.abs(params1.phi - params2.phi))
    return err/8.0 # since there are 8 terms
        

In [ ]:
def sum_matrices(Ps):
    result = np.zeros_like(Ps[0])
    for Pi in Ps:
        result += Pi
    return result

def em(x_tau, ys, Sigma_tau, params, maxsteps=10, tol=1e-1):
    i = 0; err = np.Inf
    N = ys.shape[0]
    N_days = int(N/T)
    
    while i < maxsteps or err < tol:
        
        Ps = [] # this is an array of P_t in REVERSE order
        P_minuses = [] # this is an array of P_{t|t-1} in REVERSE order, e.g. P_{N|N-1} is first and P_{1|0} is last
        xs = []
        
        Sigma_tau_minus_N = Sigma_tau # IDK how to initialize this
        for tau in range(N-1, -1, -1):
            x_tau_n, Sigma_tau_n, L_tau, x_ts, Sigma_ts = kalman_smoothing(x_tau, ys[0:tau+1], Sigma_tau, params)
            x_tau = x_tau_n
            P_tau = Sigma_tau_n + x_tau_n @ x_tau_n.T
            
            x_tau_minus_n, _, L_tau_minus, _, _ = kalman_smoothing(x_tau, ys[0:tau], Sigma_tau, params)
            # This is line 7 of Algorithm 3 which computes Sigma_{tau, tau-1|N}
            # Note that Sigma_{tau, tau-1|N} has a dependency on Sigma_{tau+1, tau|N} which makes sense except
            # we do not know what to initialize it to
            Sigma_tau_minus_N = Sigma_ts[tau-1] @ L_tau_minus.T + \
                                L_tau @ (Sigma_tau_minus_N - params.A(tau) @ Sigma_ts[tau]) @ L_tau_minus.T
            P_tau_minus = Sigma_tau_minus_N + x_tau_n @ x_tau_minus_n
            
            xs.append(x_tau)
            Ps.append(P_tau)
            P_minuses.append(P_tau_minus)
        Ps.reverse()
        P_minuses.reverse()
        print("first step done")
        
        # now we have a list of P_t and P_{t|t-1}
        pi = x_tau # equation 17
        Sigma = Ps[-1] - x_tau @ x_tau.T # equation 18

        P_sum = sum_matrices([Ps[T*i + 1] for i in range(N_days)]) # for equation 19
        P_minus_sum = sum_matrices([P_minuses[T*i + 1] for i in range(N_days)]) # for equation 19
        a_eta = P_minus_sum[0:T, 0:T] @ np.linalg.inv(P_sum[0:T,0:T])
        
        P_sum2 = sum_matrices([Ps[i] for i in range(2, N)]) # for equation 20
        P_minus_sum2 = sum_matrices([P_minuses[i] for i in range(1, N-1)]) # for equation 20
        a_mu = P_minus_sum2[T+1:, T+1:] @ np.linalg.inv(P_sum2[T+1:, T+1:])
        
        sigma_eta_sq = np.zeros((T,T)) # equation 21
        for i in range(N_days):
            t = i*T+1
            sigma_eta_sq += (Ps[t] + a_eta**2.0 * Ps[t-1] - 2.0 * a_eta * P_minuses[t])[0:T,0:T]
        sigma_eta_sq *= 1.0/(N_days + 1.0)
        
        sigma_mu_sq = np.zeros((T,T)) # equation 22
        for t in range(2, N):
            sigma_mu_sq = (Ps[t] + a_mu**2.0 * Ps[t-1] - 2.0 * a_mu * P_minuses[t])[T+1:, T+1:]
        sigma_mu_sq *= 1.0/(N - 1.0)
        
        r = 0.0
        for t in range(N): # equation 23, probably
            r += ys[t]**2 + np.sum(P[t]) - 2.0*ys[t] * np.sum(xs[t]) + \
                (params.phi[t%T])**2.0 - 2.0 * ys[t] * params.phi[t%T] + \
                2.0*params.phi[t%T] * np.sum(xs[t])
        r *= 1.0/N
        
        phi = np.zeros(N_days)
        for i in range(N_days):
            phi += ys[i*T:(i+1)*T] - C@xs[t]
        phi /= 1.0/N_days
        print("second step done")
        params1 = Params(pi, Sigma, a_eta, a_mu, sigma_eta_sq, sigma_mu_sq, r, phi)
        err = compare(params, params1)
        params = params1
        print(i, err)
        
    return params
            

In [ ]:
em(x_1, ys[1:N_train], Sigma_1, theta, maxsteps=10, tol=1e-1)

## Test Kalman filter with given params

In [ ]:
N = df["Log_Turnover"].size
x_1

In [ ]:
y_t = df.iloc[0:T]["Log_Turnover"]
x_t = np.reshape(np.array([y_t/2, y_t/2]), 2*T)
xs = [x_t,]

Sigma_t = np.identity(2*T)
sigmas = [Sigma_t,]

ys = [y_t]

for i in range(int(N/T)):
    y_t = df.iloc[i*T:(i+1)*T]["Log_Turnover"]
    x_plus, Sigma_plus = kalman_filtering(i, x_t, y_t, Sigma_t, theta)
    xs.append(x_plus)
    sigmas.append(Sigma_plus)
    ys.append(C@x_plus)



In [ ]:
errs = [np.mean(np.square(ys[i] - df.iloc[i*T:(i+1)*T]["Log_Turnover"])) for i in range(int(N/T))]

In [ ]:
plt.semilogy(errs, label="Prediction error")
plt.xlabel("Day of year"); plt.ylabel("MSE error")
plt.legend()
plt.savefig("kalman_errors_year.pdf")

In [ ]:
plt.semilogy(np.exp(ys[-1]),label="Predicted", linestyle="--", color="cornflowerblue")
plt.plot(np.exp(df.iloc[-T:]["Log_Turnover"]), label="True", color="black")

daily_labels = ["9:30", "10:30", "11:30", "12:30", "13:30", "14:30", "15:30",]
plt.xticks(ticks=range(0, N_day, 2), labels=daily_labels)
plt.legend(); plt.ylabel("Normalized turnover")
plt.savefig("kalman_prediction.pdf")

In [ ]:
np.mean(np.square(ys[-1] - df.iloc[-T:]["Log_Turnover"]))